# Webexport
!! tippecanoe installieren !!

```
brew install tippecanoe
```

In [9]:
import geopandas as gpd
import pandas as pd
import json
import os
import consts
import matplotlib.colors as mcolors
from utils import categorize_time_to_half_hour, categorize_time_to_half_hour_numerical, dt_to_time, time_to_time, categorize_time_to_6min_numerical, categorize_time_to_6min
import pyperclip

In [10]:
# Load timedata
time_data = json.load(open(consts.PATH_TIMEDATA_HEX, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(consts.PATH_HEXAGON)

In [11]:
# Serialize Timedata
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'latest_sunrise_current': x['sunrise']['current']['latest'],
    'latest_sunrise_summertime': x['sunrise']['summer']['latest'],
    'latest_sunrise_summertime_day': x['sunrise']['summer']['latest_day'],
    'latest_sunrise_wintertime': x['sunrise']['winter']['latest'],
    'earliest_sunset_current': x['sunset']['current']['earliest'],
    'earliest_sunset_summertime': x['sunset']['summer']['earliest'],
    'earliest_sunset_summertime_day': x['sunset']['summer']['earliest_day'],
    'earliest_sunset_wintertime': x['sunset']['winter']['earliest']
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert strings to time
df_timedata['latest_sunrise_current'] = df_timedata['latest_sunrise_current'].apply(time_to_time)
df_timedata['latest_sunrise_summertime'] = df_timedata['latest_sunrise_summertime'].apply(time_to_time)
df_timedata['latest_sunrise_wintertime'] = df_timedata['latest_sunrise_wintertime'].apply(time_to_time)
df_timedata['earliest_sunset_current'] = df_timedata['earliest_sunset_current'].apply(time_to_time)
df_timedata['earliest_sunset_summertime'] = df_timedata['earliest_sunset_summertime'].apply(time_to_time)
df_timedata['earliest_sunset_wintertime'] = df_timedata['earliest_sunset_wintertime'].apply(time_to_time)

# Define how detailed the output should be (half hour? 5min?)
categorizes_function = categorize_time_to_half_hour
categorizes_numerical_function = categorize_time_to_half_hour_numerical
# categorizes_function = categorize_time_to_6min
# categorizes_numerical_function = categorize_time_to_6min_numerical

def calculate_event(event):

    # Current System
    df_timedata[f'{event}_current_numerical'] = df_timedata[f'{event}_current'].apply(categorizes_numerical_function)

    # Everything on Summertime
    # df_timedata[f'{event}_summertime_category'] = df_timedata[f'{event}_summertime'].apply(categorizes_function)
    df_timedata[f'{event}_summertime_numerical'] = df_timedata[f'{event}_summertime'].apply(categorizes_numerical_function)
    
    # Everything on Wintertime
    # df_timedata[f'{event}_wintertime_category'] = df_timedata[f'{event}_wintertime'].apply(categorizes_function)
    df_timedata[f'{event}_wintertime_numerical'] = df_timedata[f'{event}_wintertime'].apply(categorizes_numerical_function)

calculate_event('latest_sunrise')    
calculate_event('earliest_sunset')    

# Merge Dataframes
gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')
gdf_grid = gdf_grid.drop(columns=['nuts_id'])

## Export GeoJSON

In [12]:
gdf_grid.to_file(consts.PATH_GEOJSON_EXPORT)

## Create fill-color-Expression for Maptiler

In [7]:
# Wintertime
# color_scale = ["#6262DD", "#9A68FC", "#F99ED6"]
# field = 'sunrise_wintertime_numerical'

# Summertime
color_scale = ["#F25D40", "#FFAA76", "#FFE876"]
field = 'latest_sunrise_summertime_numerical'

In [8]:
# Create custom colormap
cmap = mcolors.LinearSegmentedColormap.from_list("custom_cmap", color_scale)

# Normalize
norm = mcolors.Normalize(vmin=df_timedata[field].min(), vmax=df_timedata[field].max())

# Function to get color for a given value
def get_color(value):
    return mcolors.to_hex(cmap(norm(value)))

df_timedata[field].min(), df_timedata[field].max()

(np.float64(7.5), np.float64(14.0))

In [9]:
stepper = 0
values = []
for i in range(int(df_timedata[field].min() * 10), int(df_timedata[field].max() * 10) + 1):
    value = i / 10
    values.append(str(value).replace('.0', ''))
    values.append(f'"{get_color(value)}"')
    stepper += 1


maptiler_style = """
 [
      "interpolate",
      [
        "linear"
      ],
      [
        "get",
        {field}
      ],
      """ + '\n      ,'.join(map(str, values)) + """
    ],"""

pyperclip.copy(maptiler_style)